In [1]:
import pandas as pd
import nltk

In [2]:
data = pd.read_csv("stockprice_15_04_2019-25_10_2017.csv")
nabil_data = data.loc[data['traded_companies'] == "Nabil Bank Limited"]
nabil_data["date"] = pd.to_datetime(nabil_data["date"])
nabil_data.set_index("date", inplace = True)

/home/aakalpa/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
nabil_data.tail()

,traded_companies,no_of_transaction,max_price,min_price,closing_price,traded_shares,amount,previous_closing,difference_rs
date,,,,,,,,,
2017-10-18,Nabil Bank Limited,42.0,1280.0,1264.0,1277.0,4812.0,6117000.0,1273.0,4.0
2017-10-22,Nabil Bank Limited,16.0,1285.0,1270.0,1272.0,1070.0,1368105.0,1277.0,-5.0
2017-10-23,Nabil Bank Limited,30.0,1280.0,1260.0,1265.0,1708.0,2169615.0,1272.0,-7.0
2017-10-24,Nabil Bank Limited,24.0,1270.0,1251.0,1257.0,2908.0,3659420.0,1265.0,-8.0
2017-10-25,Nabil Bank Limited,32.0,1265.0,1235.0,1235.0,2350.0,2930820.0,1257.0,-22.0


In [4]:
news_data = pd.read_csv("processed_data.csv")
news_data.tail()

,Unnamed: 0,date,news
39584,39584,2011-03-28,"['gurkha_development_bank', 'limited', 'has', ..."
39585,39585,2011-03-28,"['27', 'march,2011', ':', 'kaski', 'finance', ..."
39586,39586,2011-03-27,"['citizens_bank', 'interational', 'ltd.', 'is'..."
39587,39587,2011-03-27,"['income', 'from', 'tourism', 'declines', ':',..."
39588,39588,2011-03-27,"['lumbini', 'bank_ltd.', 'has', 'shifted', 'it..."


In [5]:
news = news_data.groupby("date")['news'].apply(list)


In [6]:
news = pd.DataFrame(news)

In [7]:
result_df = nabil_data.join(news, how = "inner")

In [8]:
result_df.head()

,traded_companies,no_of_transaction,max_price,min_price,closing_price,traded_shares,amount,previous_closing,difference_rs,news
date,,,,,,,,,,
2011-03-27,Nabil Bank Limited,14.0,1160.0,1120.0,1147.0,1426.0,1619270.0,1174.0,-27.0,"[['citizens_bank', 'interational', 'ltd.', 'is..."
2011-03-28,Nabil Bank Limited,12.0,1150.0,1110.0,1132.0,352.0,393534.0,1147.0,-15.0,"[['capital', 'merchant', 'bank', '&', 'finance..."
2011-03-29,Nabil Bank Limited,24.0,1120.0,1050.0,1097.0,1456.0,1576420.0,1132.0,-35.0,"[['bhargav', 'bikash', 'bank_ltd.', 'has', 'pu..."
2011-03-30,Nabil Bank Limited,16.0,1076.0,1030.0,1040.0,1340.0,1394926.0,1097.0,-57.0,"[['finance', 'secretary', 'khanal', 'resigns',..."
2011-03-31,Nabil Bank Limited,23.0,1109.0,1020.0,1082.0,1828.0,1947570.0,1040.0,42.0,"[['real', 'sector', 'fails', 'to', 'make', 'im..."


In [9]:
import gensim

In [10]:
model = gensim.models.KeyedVectors.load("model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: [Errno 2] No such file or directory: 'model'

In [11]:
model.get_vector("nabil")

NameError: name 'model' is not defined

In [12]:
import numpy as np
def vectorize(news):
    result = []
    for new in news:
        li = list(map(model.get_vector, new))
        result.append( np.average(li, axis = 0))
    return result
    
        

In [13]:
result_df.news = result_df.news.apply(vectorize)

NameError: name 'model' is not defined

In [14]:
np.array(result_df.head(1).news)

array([list(["['citizens_bank', 'interational', 'ltd.', 'is', 'selling', 'unsold', 'promoter_share', 'in', 'auction', 'to', 'general', 'public.', ',']", "['income', 'from', 'tourism', 'declines', ':', 'nrb', ',']", "['lumbini', 'bank_ltd.', 'has', 'shifted', 'its', 'durbarmarg', 'branch', 'to', 'narayanchour', ',', 'naxal', ',']"])],
      dtype=object)

In [15]:
result_df[["closing_price", "news"]].head()

,closing_price,news
date,,
2011-03-27,1147.0,"[['citizens_bank', 'interational', 'ltd.', 'is..."
2011-03-28,1132.0,"[['capital', 'merchant', 'bank', '&', 'finance..."
2011-03-29,1097.0,"[['bhargav', 'bikash', 'bank_ltd.', 'has', 'pu..."
2011-03-30,1040.0,"[['finance', 'secretary', 'khanal', 'resigns',..."
2011-03-31,1082.0,"[['real', 'sector', 'fails', 'to', 'make', 'im..."


In [16]:
def get_momentum(stock_cp, window):
    ''''This function calculates momentum which is defined as:
            m=ct - ct-n
            where ct = closing price of current price
                  ct-n = closing price of n days before
        Input:  stock_cp = closing price pandas series
                window = t-n
        Ouput: Momentum for each day
    '''
    momentum = stock_cp.rolling(window=window).apply(lambda x: x[-1] - x[0])
    return momentum


def get_ema(stock_cp, span=12):
    '''This function calculates exponential moving average(ema) over span=span'''
    return stock_cp.ewm(span=span).mean()


def get_macd(stock_cp, nshort=12, nlong=26):
    '''This function calculates moving average convergence/divergence given by:
            MACD = ema(nshort) - ema(nlong)
    '''
    emashort = get_ema(stock_cp, nshort)
    emalong = get_ema(stock_cp, nlong)
    macd = emashort - emalong
    return macd


def get_RSI(stock_cp, window):
    '''This function calculates Relative strength index, which is given by:
                RSI= avg(ut, ut-1, ... ,ut-n+1)/ avg(dt, dt-1, ..., dt-n+1)
                where ut = ct - ct-1 if ct>ct-1 else 0
                      dt = ct-1 - ct if ct<ct-1 else 0
    '''
    # Calculating upward and downward trends
    up_trends = stock_cp.rolling(window=2).apply(lambda y: y[1] - y[0] if y[1] > y[0] else 0)
    down_trends = stock_cp.rolling(window=2).apply(lambda y: y[0] - y[1] if y[0] > y[1] else 0)

    # Averaging upward and downward trends over period of window
    avg_uptrends = up_trends.rolling(window=window).mean()
    # print(avg_uptrends.head(30))
    avg_downtrends = down_trends.rolling(window=window).mean()
    # print(avg_downtrends.head(30))

    # Finally RSI
    RSI = avg_uptrends.divide(avg_downtrends)
    return RSI


def get_ROI(stock_cp):
    '''this function calculates average of past n days(n = window) divided by closing price'''
    temp1 = stock_cp.rolling(window=10).mean()
    temp2 = stock_cp
    return temp1 / temp2


def get_stochrsi(stock_rsi, window):
    '''This function calculates stochastic RSI which is given by:
                STOCHRSI = (RSI - min{RSIt, RSIt-1,..., RSIt-n+1}))/(max{RSIt, RSIt-1,..., RSIt-n+1} - min{RSIt, RSIt-1,..., RSIt-n+1})
    '''
    min_rsi = stock_rsi.rolling(window=window).min()
    max_rsi = stock_rsi.rolling(window=window).max()
    stochrsi = (stock_rsi - min_rsi) / (max_rsi - min_rsi)
    return stochrsi


def get_ATR(stock_cp, window):
    min_ATR = stock_cp.rolling(window=window).min()
    max_ATR = stock_cp.rolling(window=window).max()
    TR_final = max_ATR - min_ATR
    final_ATR = get_ema(TR_final, span=14)
    return TR_final


def get_ADX(stock_cp, window):
    max_ADX = stock_cp.rolling(window=window).max()
    min_ADX = stock_cp.rolling(window=window).min()
    up = max_ADX.rolling(window=window).apply(lambda x: x[-1] - x[-2])
    dw = min_ADX.rolling(window=window).apply(lambda x: x[-2] - x[-1])
    plus_DM = pd.DataFrame(np.where((up > dw) & (up > 0), up, 0))
    minus_DM = pd.DataFrame(np.where((up < dw) & (dw > 0), dw, 0))
    plus_DI = get_ema(plus_DM, span=14)
    minus_DI = get_ema(minus_DM, span=14)
    sub_ADX = plus_DI - minus_DI
    result = get_ema(sub_ADX) / sub_ADX
    return result


def get_williams(stock_cp, window):
    min_cp = stock_cp.rolling(window=window).min()
    max_cp = stock_cp.rolling(window=window).max()
    williams = (max_cp - stock_cp) / (max_cp - min_cp)
    return williams


def get_CCI(stock_cp, window):
    min_cp = stock_cp.rolling(window=window).min()
    max_cp = stock_cp.rolling(window=window).max()
    result = (max_cp + min_cp + stock_cp) / 3
    return result


def get_AVG_for_UO(stock_cp, n):
    min_ATR = stock_cp.rolling(window=n).min()
    max_ATR = stock_cp.rolling(window=n).max()
    TR_final = max_ATR - min_ATR
    bp = stock_cp - min_ATR
    numerator = bp.rolling(window=n).apply(lambda x: x[-1] + x[-2] + x[-n])
    denominator = TR_final.rolling(window=n).apply(lambda x: x[-1] + x[-2] + x[-n])
    return numerator / denominator


def get_UO(stock_cp, window):
    uo = (4 * get_AVG_for_UO(stock_cp, 7) + 2 * get_AVG_for_UO(stock_cp, 14) + get_AVG_for_UO(stock_cp, 28)) / 7
    return uo

def get_tech_indicators(stock_csv, window):
    ''' This function calculates the technical indicators and append them to the closing price in the given csv file.
        Input: stock_csv = closing price csv file
               window = no.of days to average to calculate technical indicators
        Ouput: csvfile with technical indicators appended to closing price of each day
    '''
    # Here, a pandas dataframe is made by reading csv file
    stock_df = pd.read_csv(stock_csv, index_col=0)
    stock_cp = stock_df['nepse_index']

    # Add Momentum column
    stock_df['momentum'] = get_momentum(stock_cp, window)

    # Add EMA
    stock_df['EMA'] = get_ema(stock_cp)

    # Add MACD
    stock_df['MACD'] = get_macd(stock_cp)

    # Add RSI
    stock_df['RSI'] = get_RSI(stock_cp, window)

    # Add STOCHRSI
    stock_df['STOCHRSI'] = get_stochrsi(stock_df['RSI'], window)

    stock_df['ROI'] = get_ROI(stock_cp)

    stock_df['ATR'] = get_ATR(stock_cp, window)

    stock_df['ADX'] = get_ADX(stock_cp, window)

    stock_df['williams'] = get_williams(stock_cp, window)

    stock_df['CCI'] = get_CCI(stock_cp, window)

    stock_df['UO'] = get_UO(stock_cp, window)
    return stock_df


In [17]:
stock_df = get_tech_indicators("Nepse_index.csv", 5)



/home/aakalpa/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  # Remove the CWD from sys.path while we load stuff.
/home/aakalpa/.local/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
/home/aakalpa/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True

In [18]:
stock_df = stock_df.drop([0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22])


In [19]:
stock_df = stock_df.drop(['UO'],axis =1)

In [20]:
stock_df.head(30)

,date,nepse_index,momentum,EMA,MACD,RSI,STOCHRSI,ROI,ATR,ADX,williams,CCI
23,2018-07-15,1814.430054,-84.819946,1837.585960,-6.762209,0.275102,0.000000,1.014784,125.479980,-5.770594,0.675964,1829.150024
24,2018-07-16,1846.719971,-51.119995,1839.013105,-5.540793,0.581367,0.063240,0.999918,124.069946,-5.486751,0.412026,1839.443319
25,2018-07-17,1793.609985,-17.810059,1831.936064,-8.254247,0.411728,0.214111,1.029163,72.949951,-5.208680,0.728033,1804.699992
26,2018-07-18,1793.609985,19.839966,1825.974203,-10.271957,0.803768,1.000000,1.027637,72.949951,-4.936359,0.728033,1804.699992
27,2018-07-19,1808.219971,-6.210083,1823.217136,-10.680821,1.648653,1.000000,1.015388,53.109985,-4.669757,0.724911,1816.183309
28,2018-07-22,1773.650024,-73.069946,1815.530928,-13.367648,0.534899,0.099578,1.026838,73.069946,-0.104037,1.000000,1798.006673
29,2018-07-23,1773.650024,-19.959961,1809.044512,-15.310002,0.166629,0.000000,1.019757,34.569946,0.053211,1.000000,1785.173340
30,2018-07-24,1823.780029,30.170044,1811.324363,-12.983284,1.872725,1.000000,0.987666,50.130005,2.625605,0.000000,1807.070028
31,2018-07-25,1807.260010,-0.959961,1810.696082,-12.231167,1.267176,0.645068,0.996464,50.130005,2.719903,0.329544,1801.563354
32,2018-07-26,1846.760010,73.109985,1816.266856,-8.576981,1.754356,0.930620,0.979103,73.109985,0.125100,0.000000,1822.390015


In [21]:
final = stock_df.join(result_df,how='inner')

In [22]:
final.head()

,date,nepse_index,momentum,EMA,MACD,RSI,STOCHRSI,ROI,ATR,ADX,...,traded_companies,no_of_transaction,max_price,min_price,closing_price,traded_shares,amount,previous_closing,difference_rs,news


In [ ]:
df.to_csv(fi, encoding='utf-8', index=False)